In [ ]:
import logging
import os
import uuid
from typing import List

from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_openai import ChatOpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import FieldCondition, Filter, MatchValue



In [ ]:
new_list = ['spraw gospodarki wodnej, na podstawie art. 190 ust. 9.\n\x0c©Kancelaria Sejmu    s. 155/444 \n \n2024-01-24 \n \n13. Jeżeli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym \nmowa w ust. 1, lub nie uzyskał decyzji o legalizacji urządzenia wodnego, organ \nwłaściwy w sprawach pozwoleń wodnoprawnych na wykonywanie urzą dzeń',
 'Art. 190. 1. Jeżeli urządzenie wodne zostało wykonane bez wymaganego \npozwolenia wodnoprawnego lub zgłoszenia,  właściciel tego urządzenia może \nwystąpić z  wnioskiem o  jego legalizacj ę, do którego dołącza odpowiednio \ndokumenty, o których mowa w art. 407 ust. 2 oraz w art. 422. \n2. Organ właściwy w sprawach pozwoleń wodnoprawnych na wykonywanie',
 'wodnego, o którym mowa w ust. 1, lub ograniczeniu korzystania z  tego \nurządzenia odpowiednio właściciela urządzenia wodnego lub podmiot \nkorzystający z tego urządzenia – jeżeli jest możliwe ich ustalenie, \nb) podejmuje działania w  celu wszczęcia procedury legalizacji urządzenia \nwodnego ‒ w przypadku korzystania z urządzenia wodnego wykonanego',
 'urządzenia wodne lub ich części, których pozostawienie jest niezbędne do \nkształtowania zasobów wodnych, przechodzą, bez odszkodowania, na własność \nwłaściciela wód, chyba że  właściciel urządzenia wodnego, w terminie 6 miesięcy \nod dnia, w którym: \n1) pozwolenie wodnoprawne wygasło – w przypadku upływu terminu, na który \npozwolenie to zostało wydane,',
 'bez wymaganego pozwolenia wodnoprawnego lub zgłoszenia \nwodnoprawnego, \nc) podejmuje działania w  celu wszczęcia p ostępowania na podstawie \nart. 343 ust. 1 lub 6  ‒ w przypadku korzystania z wód przez zakład bez \nwymaganego pozwolenia wodnoprawnego lub zgłoszenia \nwodnoprawnego. \n4. Właściciel urządzenia wodnego, o  którym mowa w  ust. 1, lub podmiot']

In [ ]:
from sentence_transformers import CrossEncoder

# Load a pre-trained CrossEncoder model
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Predict scores for a pair of sentences
 
scores = model.predict([
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[0]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[1]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[2]),
	("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[3]),
    ("Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?", new_list[4]),
])

# => ar

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

model_path = "radlab/polish-cross-encoder"
model = CrossEncoder(model_path)


questions = ["Co się stanie jeśli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym mowa w ust. 1?"]
answers = new_list
for question in questions:
    context_with_question = [(s, question) for s in answers]
    results = sorted(
        {
            idx: r for idx, r in enumerate(model.predict(context_with_question))
        }.items(),
        key=lambda x: x[1],
        reverse=True,
    )

    print(f"QUESTION: {question}")
    print("ANSWERS (sorted):")
    for idx, score in results:
        print(f"\t[{score}]\t{idx}")
    print("")

In [18]:
client = QdrantClient(url=os.getenv("QDRANT_ADDRESS"))
multi_query_list=['Pytanie1: Czy właściciel urządzenia wodnego złożył wniosek zgodnie z ust. 1?',
				  'Pytanie2: Jakie są konsekwencje, jeśli właściciel urządzenia wodnego nie złoży wniosku, o którym mowa w ust. 1?',
				  'Pytanie3: Czy istnieją przypadki, w których właściciel urządzenia wodnego nie musi składać wniosku zgodnie z ust. 1?']
response = []
collection_name = os.environ["COLLECTION_NAME"]
def vectorize_user_query(query: str) -> List[float]:
    """Embeds user_query vector"""
    embedder = NVIDIAEmbeddings(model=os.environ["EMBEDDER"])
    return embedder.embed_query(query)


for index, query in enumerate(multi_query_list):
	clean_query = query.replace(f"Pytanie{index + 1}: ", "")
	logging.info(clean_query)

	points = client.query_points(
		collection_name=collection_name,
		query=vectorize_user_query(query=clean_query),
		limit=5,
		with_payload=True,
	).points

	logging.info("retrive tool bet answers: ", points)
	response.extend([(clean_query, point) for point in points])
print(response)
response = sorted(response, key=lambda p: p[1].score, reverse=True)
seen_id = set()
filtered_list = []

for question, point in response:
	if point.id not in seen_id:
		seen_id.add(point.id)
		filtered_list.append((question, point.payload['content']['page_content']))


client.close()

[('Czy właściciel urządzenia wodnego złożył wniosek zgodnie z ust. 1?', ScoredPoint(id=938, version=938, score=0.6628634, payload={'content': {'metadata': {'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-24T10:08:08+01:00', 'title': 'Akt prawny', 'author': 'Władysław Baksza', 'moddate': '2024-01-24T10:08:08+01:00', 'source': './data/prawod_wodne.pdf', 'total_pages': 444}, 'page_content': 'Art. 190. 1. Jeżeli urządzenie wodne zostało wykonane bez wymaganego \npozwolenia wodnoprawnego lub zgłoszenia,  właściciel tego urządzenia może \nwystąpić z  wnioskiem o  jego legalizacj ę, do którego dołącza odpowiednio \ndokumenty, o których mowa w art. 407 ust. 2 oraz w art. 422. \n2. Organ właściwy w sprawach pozwoleń wodnoprawnych na wykonywanie'}}, vector=None, shard_key=None, order_value=None)), ('Czy właściciel urządzenia wodnego złożył wniosek zgodnie z ust. 1?', ScoredPoint(id=1790, version=1790, score=0.65414286, payload={'content': {'metadat

In [19]:
filtered_list

[('Czy istnieją przypadki, w których właściciel urządzenia wodnego nie musi składać wniosku zgodnie z ust. 1?',
  'Art. 190. 1. Jeżeli urządzenie wodne zostało wykonane bez wymaganego \npozwolenia wodnoprawnego lub zgłoszenia,  właściciel tego urządzenia może \nwystąpić z  wnioskiem o  jego legalizacj ę, do którego dołącza odpowiednio \ndokumenty, o których mowa w art. 407 ust. 2 oraz w art. 422. \n2. Organ właściwy w sprawach pozwoleń wodnoprawnych na wykonywanie'),
 ('Jakie są konsekwencje, jeśli właściciel urządzenia wodnego nie złoży wniosku, o którym mowa w ust. 1?',
  'spraw gospodarki wodnej, na podstawie art. 190 ust. 9.\n\x0c©Kancelaria Sejmu    s. 155/444 \n \n2024-01-24 \n \n13. Jeżeli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym \nmowa w ust. 1, lub nie uzyskał decyzji o legalizacji urządzenia wodnego, organ \nwłaściwy w sprawach pozwoleń wodnoprawnych na wykonywanie urzą dzeń'),
 ('Czy istnieją przypadki, w których właściciel urządzenia wodnego nie musi

In [20]:
cross_encoder_model = CrossEncoder(os.environ["CROSS_ENCODER_MODEL"])
results = sorted(
	{
		idx: r for idx, r in enumerate(cross_encoder_model.predict(filtered_list))
	}.items(),
	key=lambda x: x[1],
	reverse=True,
)


d:\Studia\AquaRAG\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [25]:
top_documents = "\n\n".join(
    filtered_list[index][1] for index, _ in results[:5]
)
top_documents

'wodnego, o którym mowa w ust. 1, lub ograniczeniu korzystania z  tego \nurządzenia odpowiednio właściciela urządzenia wodnego lub podmiot \nkorzystający z tego urządzenia – jeżeli jest możliwe ich ustalenie, \nb) podejmuje działania w  celu wszczęcia procedury legalizacji urządzenia \nwodnego ‒ w przypadku korzystania z urządzenia wodnego wykonanego\n\nspraw gospodarki wodnej, na podstawie art. 190 ust. 9.\n\x0c©Kancelaria Sejmu    s. 155/444 \n \n2024-01-24 \n \n13. Jeżeli właściciel urządzenia wodnego nie wystąpił z wnioskiem, o którym \nmowa w ust. 1, lub nie uzyskał decyzji o legalizacji urządzenia wodnego, organ \nwłaściwy w sprawach pozwoleń wodnoprawnych na wykonywanie urzą dzeń\n\nwodnych nakłada na właściciela tego urządzenia, w drodze decyzji, obowiązek \nlikwidacji urządzenia, ustalając warunki i termin wykonania tego obowiązku. \n14. Jeżeli właściciel urządzenia wodnego nie uzyskał decyzji o legalizacji \nurządzenia wodnego, a likwidacja urządzenia jest niemożliwa ze wzglę

In [ ]:
import os
from typing import List, Tuple

from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, PointStruct, VectorParams

def ReadPdf(path_to_pdf: str) -> List[Document]:
    
    """
    This function returns list of langchain documents.
    Loading whole file as one document in order to split it with chunk overlap.
    """
    try:
        loader = PyPDFLoader(file_path=path_to_pdf, mode="page", extraction_mode="layout")
        print("Loading pdf file...")
        document = loader.load()
        return document

    except ValueError as e:
        print(
            "\033[91mValueError in ReadPdf: Provided path does not lead to a file: \033[0m",
            e,
        )
        return []
document = ReadPdf("./data/prawod_wodne.pdf")

Loading pdf file...


In [148]:
document

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2024-01-24T10:08:08+01:00', 'title': 'Akt prawny', 'author': 'Władysław Baksza', 'moddate': '2024-01-24T10:08:08+01:00', 'source': './data/prawod_wodne.pdf', 'total_pages': 444, 'page': 0, 'page_label': '1'}, page_content='©Kancelaria Sejmu    s. 1/444 \n \n2024-01-24 \n \n \nDz. U. 2017 poz. 1566  \n \n \nU S T AWA \nz dnia 20 lipca 2017 r. \nPrawo wodne1) \nDZIAŁ I \nZasady ogólne \nRozdział 1 \nPrzepisy ogólne \nArt. 1. Ustawa reguluje gospodarowanie wodami zgodnie z  zasadą \nzrównoważonego rozwoju, w  szczególności kształ towanie i  ochronę zasobów \nwodnych, korzystanie z wód oraz zarządzanie zasobami wodnymi. \n                                                           \n1) Niniejsza ustawa w zakresie swojej regulacji wdraża: \n1) dyrektywę Rady 91/271/EWG z  dnia 21  maja 1991  r. dotyczącą oczyszczania ścieków \nkomunalnych (Dz. Urz. WE L 135 z  30.05.1991, str. 40, z pó

In [147]:
splitter = RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=60,
        length_function=len,
        is_separator_regex=True,
        separators=["Art\.\s*\d*", "\\n\d(\.|\))"],
    )

doc = document
doc_splitted = splitter.split_documents(documents=doc)
doc_splitted_txt = [doc.page_content for doc in doc_splitted]

IndexError: list index out of range

In [145]:
len(doc_splitted_txt)

946

In [140]:
max_len = -1
min_len = 30000
for i in doc_splitted_txt:
    max_len = max(max_len, len(i))
    min_len = min(max_len, len(i))
print(max_len, min_len)

2610 724


In [142]:
doc_splitted_txt

['©Kancelaria Sejmu    s. 1/444 \n \n2024-01-24 \n \n \nDz. U. 2017 poz. 1566  \n \n \nU S T AWA \nz dnia 20 lipca 2017 r. \nPrawo wodne1) \nDZIAŁ I \nZasady ogólne \nRozdział 1 \nPrzepisy ogólne',
 'Art. 1. Ustawa reguluje gospodarowanie wodami zgodnie z  zasadą \nzrównoważonego rozwoju, w  szczególności kształ towanie i  ochronę zasobów \nwodnych, korzystanie z wód oraz zarządzanie zasobami wodnymi. \n                                                           \n1) Niniejsza ustawa w zakresie swojej regulacji wdraża: \n1) dyrektywę Rady 91/271/EWG z  dnia 21  maja 1991  r. dotyczącą oczyszczania ścieków \nkomunalnych (Dz. Urz. WE L 135 z  30.05.1991, str. 40, z późn. zm. – Dz. Urz. UE Polskie \nwydanie specjalne, rozdz. 15, t. 2, str. 26, Dz. Urz. WE L 67 z 07.03.1998, str. 29 – Dz. Urz. \nUE Polskie wydanie specjalne, rozdz. 15,  t. 4, str. 27, Dz. Urz. UE L 284 z 31.10.2003, str. \n1 – Dz. Urz. UE Polskie wydanie specjalne, rozdz. 1, t.  4, str.  447, Dz.  Urz. UE  L 311 \nz 21.11.2

In [143]:
splitter = RecursiveCharacterTextSplitter(
	chunk_size=600,
	chunk_overlap=60,
	length_function=len,
	is_separator_regex=True,
	separators=["(Art\.\s*\d* | \\n\d(\.|\)))"],
)
doc_splitted_2 = splitter.split_documents(documents=doc_splitted)
doc_splitted_txt_2 = [doc.page_content for doc in doc_splitted_2]
#(\\n\d(\.|\)) | Art\.\d)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [115]:
doc_splitted_txt_2

['©Kancelaria Sejmu    s. 1/444 \n \n2024-01-24 \n \n \nDz. U. 2017 poz. 1566  \n \n \nU S T AWA \nz dnia 20 lipca 2017 r. \nPrawo wodne1) \nDZIAŁ I \nZasady ogólne \nRozdział 1 \nPrzepisy ogólne',
 'Art. 1. Ustawa reguluje gospodarowanie wodami zgodnie z  zasadą \nzrównoważonego rozwoju, w  szczególności kształ towanie i  ochronę zasobów \nwodnych, korzystanie z wód oraz zarządzanie zasobami wodnymi. \n                                                           \n1) Niniejsza ustawa w zakresie swojej regulacji wdraża: \n1) dyrektywę Rady 91/271/EWG z  dnia 21  maja 1991  r. dotyczącą oczyszczania ścieków \nkomunalnych (Dz. Urz. WE L 135 z  30.05.1991, str. 40, z późn. zm. – Dz. Urz. UE Polskie \nwydanie specjalne, rozdz. 15, t. 2, str. 26, Dz. Urz. WE L 67 z 07.03.1998, str. 29 – Dz. Urz. \nUE Polskie wydanie specjalne, rozdz. 15,  t. 4, str. 27, Dz. Urz. UE L 284 z 31.10.2003, str. \n1 – Dz. Urz. UE Polskie wydanie specjalne, rozdz. 1, t.  4, str.  447, Dz.  Urz. UE  L 311 \nz 21.11.2

In [112]:
len(doc_splitted_txt_2)

564